In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import glob
import json
import pickle as pkl
import ast
import os

In [2]:
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
LEVEL1_FOLDERS = ['facebook_linkposts', 'facebook_linkposts_redo1', 'facebook_linkposts_redo2']
allfiles = []

for l in LEVEL1_FOLDERS:
    l1path = '/data/shruti/ONR/big_data/' + l + '/*'
    LEVEL2_FOLDERS = glob.glob(l1path)
    for l2 in LEVEL2_FOLDERS:
        l2path = l2 + "/*.json"
        fnames = glob.glob(l2path)
        for f in fnames:
            if os.stat(f).st_size == 0:
                fakevar=1
            else:
                allfiles.append(f)


In [4]:
len(allfiles)

3101

In [5]:
# for f in allfiles:
#     print(f)
#     with open(f, "r") as jfile:
#         for line in jfile:
#             job = json.loads(line)

In [6]:
allfiles = list(set(allfiles))
len(allfiles)

3101

In [7]:
# job.keys()

In [8]:
def get_contents(fileName):
    dictlist = []
    with open(fileName, "r") as jfile:
        for line in jfile:
            tempdict = {}
            try:
                job = json.loads(line)
            except:
                job = None
            if job!=None:    
                if "date" in job:
                    tempdict['date'] = job['date']
                    if "title" in job:
                        tempdict['title'] = job['title']
                    if "caption" in job:
                        tempdict['caption'] = job['caption']
                    if "description" in job:
                        tempdict['description'] = job['description']
                    if "description" in job:
                        tempdict['description'] = job['description']
                    if "message" in job:
                        tempdict['message'] = job['message']
                    if "link" in job:
                        tempdict['link'] = job['link']
                    if "postUrl" in job:
                        tempdict['postUrl'] = job['postUrl']
                    if "score" in job:
                        tempdict['score'] = job['score']

                    if "account" in job:
                        try:
                            tempdict['handle'] = job['account']['handle']
                        except:
                            fakevar=1

                        try:
                            tempdict['platformId'] = job['account']['platformId']
                        except:
                            fakevar=1

                        try:
                            tempdict['id'] = job['account']['id']
                        except:
                            fakevar=1

                    dictlist.append(tempdict)

    if len(dictlist) > 0:
        return dictlist
                
       
    
                        
            
            

In [9]:
filcontent = pd.DataFrame()
filcontent['fileNames'] = allfiles
filcontent['content'] = filcontent['fileNames'].parallel_apply(lambda x: get_contents(x))

In [10]:
len(filcontent.dropna(subset=['content']))

3101

In [11]:
def get_contentLength(content):
    try:
        return len(content)
    except:
        return 0
    
filcontent['contentLength'] = filcontent['content'].parallel_apply(lambda x: get_contentLength(x))

In [12]:
filcontent.head(5)

,fileNames,content,contentLength
0,/data/shruti/ONR/big_data/facebook_linkposts_r...,"[{'date': '2021-05-07 15:18:16', 'title': 'औरं...",28
1,/data/shruti/ONR/big_data/facebook_linkposts_r...,"[{'date': '2021-07-01 11:15:25', 'title': 'Wat...",18308
2,/data/shruti/ONR/big_data/facebook_linkposts_r...,"[{'date': '2021-05-01 03:14:04', 'title': 'अचा...",11783
3,/data/shruti/ONR/big_data/facebook_linkposts/b...,"[{'date': '2021-08-01 01:52:32', 'title': 'শিক...",738
4,/data/shruti/ONR/big_data/facebook_linkposts_r...,"[{'date': '2021-01-28 12:00:32', 'title': 'আম ...",540


In [13]:
invalidfiles = filcontent.loc[filcontent['contentLength']==0]['fileNames'].tolist()

In [14]:
invalidfiles

[]

In [15]:
jsonList = []

for idx, row in filcontent.iterrows():
    for c in row['content']:
        jsonList.append(c)

In [16]:
len(jsonList)

11694115

In [17]:
fdata = pd.DataFrame(jsonList)

In [18]:
fdata.head()

,date,title,caption,description,message,link,postUrl,score,handle,platformId,id
0,2021-05-07 15:18:16,औरंगाबाद जिल्ह्यातील रिक्त पदे भरण्याची प्रक्र...,navarashtra.com,सदर याचिकेची पहिली सुनावणी दिनांक ०४ मे २०२१ र...,https://www.navarashtra.com/aurangabad-news-ma...,https://www.navarashtra.com/aurangabad-news-ma...,https://www.facebook.com/719366294805607/posts...,-1.546592,imtiaz.jaleel,100044411200146,6611956
1,2021-05-07 05:36:34,"खाजगी रुग्णालयांमधील लाखोंची लूट थांबवा, माण -...",navarashtra.com,सध्याचे वास्तव पहाता खाजगी रुग्णालये रुग्णाला ...,"खाजगी रुग्णालयांमधील लाखोंची लूट थांबवा, माण –...",https://www.navarashtra.com/satara-news-marath...,https://www.facebook.com/104963460865558/posts...,0.000000,JaykumarGoreSpeaks,104963460865558,17548944
2,2021-05-11 16:17:52,लोकल प्रवासाची मागणी फेटाळली; सहकारी बँक युनिय...,navarashtra.com,"लोकल ट्रेन, मेट्रो आणि मोनोमधील प्रवासावर घालण...",NaN,https://www.navarashtra.com/mumbai-news-marath...,https://www.facebook.com/groups/27747221305464...,-12.500000,NaN,277472213054649,14493955
3,2021-05-11 03:10:32,जबाबदारीचे गांभीर्य कळण्यासाठी टीका; परिस्थिती...,navarashtra.com,"जगभरातील आरोग्यविषयक उपक्रम, संशोधन आणि आरोग्य...",NaN,https://www.navarashtra.com/featured-stories/c...,https://www.facebook.com/groups/22908796412365...,-0.941176,NaN,229087964123654,9065866
4,2021-05-11 09:05:00,लग्नाच्या दिवशीच वधू परिवारासह झाली गायब; वरान...,navarashtra.com,ही घटना सात एप्रिलची असून हबीबगंज सरकंडी आणि प...,NaN,https://www.navarashtra.com/india-news-marathi...,https://www.facebook.com/groups/27747221305464...,-25.000000,NaN,277472213054649,14493955


In [19]:
fdata.to_csv("/data/shruti/ONR/small_data/facebook_linkpost_text.csv")

In [20]:
data = fdata.dropna(subset=['title'])
len(data)

10692105

In [21]:
data = data.dropna(subset=['handle'])

In [22]:
len(data)

9033851

In [23]:
data = data.drop_duplicates(subset=['handle','title'])
len(data)

7075278

In [24]:
data['is_duplicated'] = data.duplicated(subset=['title'], keep=False)
data.head()

<ipython-input-24-b815f52a03a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['is_duplicated'] = data.duplicated(subset=['title'], keep=False)


,date,title,caption,description,message,link,postUrl,score,handle,platformId,id,is_duplicated
0,2021-05-07 15:18:16,औरंगाबाद जिल्ह्यातील रिक्त पदे भरण्याची प्रक्र...,navarashtra.com,सदर याचिकेची पहिली सुनावणी दिनांक ०४ मे २०२१ र...,https://www.navarashtra.com/aurangabad-news-ma...,https://www.navarashtra.com/aurangabad-news-ma...,https://www.facebook.com/719366294805607/posts...,-1.546592,imtiaz.jaleel,100044411200146,6611956,False
1,2021-05-07 05:36:34,"खाजगी रुग्णालयांमधील लाखोंची लूट थांबवा, माण -...",navarashtra.com,सध्याचे वास्तव पहाता खाजगी रुग्णालये रुग्णाला ...,"खाजगी रुग्णालयांमधील लाखोंची लूट थांबवा, माण –...",https://www.navarashtra.com/satara-news-marath...,https://www.facebook.com/104963460865558/posts...,0.000000,JaykumarGoreSpeaks,104963460865558,17548944,False
28,2021-07-01 11:15:25,Watch how Hema Malini screamed like a kid on s...,hindustantimes.com,NaN,This video is proof of the sweetest bond betwe...,https://www.hindustantimes.com/entertainment/b...,https://www.facebook.com/111445058579/posts/10...,75.561947,hindustantimes,111445058579,33887,False
29,2021-07-01 01:43:47,"Steve Waugh beats Virat Kohli, Ricky Ponting, ...",hindustantimes.com,"Steve Waugh, who captained Australia in 57 Tes...","Humbled, thank you...",https://www.hindustantimes.com/cricket/steve-w...,https://www.facebook.com/100044384051423/posts...,12.641566,SteveWaughOfficial,100044384051423,1167655,True
31,2021-07-01 09:39:29,"আরও একধাপ এগিয়ে গেল বিমানবন্দর মেট্রোর কাজ, শে...",bangla.hindustantimes.com,"প্রায় ২৫ ঘণ্টার চেষ্টায় ৪০ মিটার লম্বা, ৩৭ মিট...","প্রায় ২৫ ঘণ্টার চেষ্টায় ৪০ মিটার লম্বা, ৩৭ মিট...",https://bangla.hindustantimes.com/bengal/kolka...,https://www.facebook.com/107751030630685/posts...,39.376238,hindustantimesbangla,107751030630685,11589589,False


In [25]:
dup_data = data.loc[data['is_duplicated']==True]
dup_data.head()

,date,title,caption,description,message,link,postUrl,score,handle,platformId,id,is_duplicated
29,2021-07-01 01:43:47,"Steve Waugh beats Virat Kohli, Ricky Ponting, ...",hindustantimes.com,"Steve Waugh, who captained Australia in 57 Tes...","Humbled, thank you...",https://www.hindustantimes.com/cricket/steve-w...,https://www.facebook.com/100044384051423/posts...,12.641566,SteveWaughOfficial,100044384051423,1167655,True
32,2021-07-01 16:25:24,‘Joy of getting out of quarantine’: BCCI share...,hindustantimes.com,The tour begins from July 13 in Colombo and be...,The tour begins from July 13 in Colombo and In...,https://www.hindustantimes.com/cricket/joy-of-...,https://www.facebook.com/111445058579/posts/10...,14.517699,hindustantimes,111445058579,33887,True
35,2021-07-01 04:31:50,Taapsee Pannu recalls dad's awkwardness at wat...,hindustantimes.com,Taapsee Pannu has recalled how awkward things ...,Taapsee Pannu has recalled how awkward things ...,https://www.hindustantimes.com/entertainment/b...,https://www.facebook.com/111445058579/posts/10...,9.973451,hindustantimes,111445058579,33887,True
36,2021-07-01 04:50:29,Ashish Chowdhry bids adieu to his 'guiding lig...,hindustantimes.com,Ashish Chowdhry penned a note and shared pictu...,Ashish Chowdhry penned a note and shared pictu...,https://www.hindustantimes.com/entertainment/b...,https://www.facebook.com/228527747249282/posts...,45.500000,HTMumbai,228527747249282,938761,True
41,2021-07-01 08:40:41,"'Had Hardik been around, there wouldn't have b...",hindustantimes.com,Prasad explained the reason Ravindra Jadeja pl...,MSK Prasad stated that there was a reason Ravi...,https://www.hindustantimes.com/cricket/had-har...,https://www.facebook.com/111445058579/posts/10...,7.243363,hindustantimes,111445058579,33887,True


In [26]:
len(dup_data)

2257721

In [27]:
unique_texts = list(set(dup_data['title'].tolist()))

In [28]:
len(unique_texts)

834330

In [31]:
from random import sample

sample_unique = sample(unique_textsque_texts, 100000)

smaldup_datadata = dup_data.loc[dup_data['title'].isin(sample_unique)]

In [32]:
def assign_unique_group(text):
    uid = sample_unique.index(text)
    return uid

In [33]:
smaldup_datadata['text_id'] = smaldup_datadata['title'].parallel_apply(lambda x: assign_unique_group(x))

In [34]:
smaldup_datadata.head()

In [ ]:
connections = smaldup_datadata.groupby('text_id')['handle'].apply(list).reset_index()
connections.head()